In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy import ndimage

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [3]:
wo_2016 = gpd.read_file('/home/philipp/Data/change_detection/GIS_data/wo_2016/wo_2016.shp')
wo_2020 = gpd.read_file('/home/philipp/Data/change_detection/GIS_data/wo_2020/wo_2020.shp')

In [4]:
wo_2016.head()

,fb,fr,abt,uabt,teilfl,admin,length,area,fly_date,year_fly,...,cut_sum,dr,bp,zp,ru,sg,ma,year_fe,age_diff,geometry
0,171,1,65535,0,1,529,195.920105,1553.419067,2015-08-30,2015,...,0.0,0,0,0,0,0,0,0,0,"POLYGON ((616346.810 493670.595, 616342.890 49..."
1,171,1,65535,0,1,529,1125.586060,35217.652344,2015-08-30,2015,...,0.0,0,0,0,0,0,0,0,0,"POLYGON ((615325.040 492902.555, 615263.980 49..."
2,171,1,65535,0,1,529,946.503113,24184.699219,2015-08-30,2015,...,0.0,0,0,0,0,0,0,0,0,"POLYGON ((608107.790 487993.105, 608108.120 48..."
3,171,1,65535,0,1,529,319.914032,5266.038574,2015-08-30,2015,...,0.0,0,0,0,0,0,0,0,0,"POLYGON ((614628.190 492540.735, 614612.100 49..."
4,171,1,506,3,1,529,2402.546875,11624.655273,2015-08-30,2015,...,0.0,0,0,0,0,0,0,2009,6,"POLYGON ((617139.640 493799.045, 617151.260 49..."


In [5]:
wo_2016.columns

Index(['fb', 'fr', 'abt', 'uabt', 'teilfl', 'admin', 'length', 'area',
       'fly_date', 'year_fly', 'WO', 'fb_sap', 'fr_sap', 'abt_sap', 'uabt_sap',
       'teilfl_sap', 'start_term', 'uz', 'non_forest', 'economy', 'ww_sw',
       'sw_type', 'sea_level', 'exp', 'slope', 'site_type', 'veg_type',
       'growth_are', 'age', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI', 'DG', 'SK',
       'mass_ha', 'mass_ha_lh', 'mass_ha_nh', 'mass_tot', 'mass_tot_l',
       'mass_tot_n', 'cut_area', 'cut_lh', 'cut_nh', 'cut_sum', 'dr', 'bp',
       'zp', 'ru', 'sg', 'ma', 'year_fe', 'age_diff', 'geometry'],
      dtype='object')

In [6]:
# filter data for
# FB 173
# FR 6,7,8,9

wo_old = wo_2016.loc[(wo_2016['fb']==176) & (wo_2016['fr'].isin([6,7,8])),\
                     ['WO', 'fb', 'fr', 'age', 'mass_ha', 'geometry']]
wo_new = wo_2020.loc[(wo_2020['fb']==176) & (wo_2020['fr'].isin([6,7,8])),\
                     ['WO', 'fb', 'fr', 'age', 'mass_ha', 'geometry']]

In [7]:
wo_old

,WO,fb,fr,age,mass_ha,geometry
163000,176060-101,176,6,0,0.000000,"POLYGON ((445894.970 406483.295, 445892.530 40..."
163001,176060-101,176,6,0,0.000000,"POLYGON ((445218.230 404334.245, 445224.260 40..."
163002,176060-101,176,6,0,0.000000,"POLYGON ((447007.410 406902.505, 446999.280 40..."
163003,176060-101,176,6,0,0.000000,"POLYGON ((443761.980 408475.555, 443786.310 40..."
163004,176060-101,176,6,0,0.000000,"POLYGON ((444079.300 407581.335, 444076.640 40..."
...,...,...,...,...,...,...
170258,17608403A3,176,8,26,9.180000,"POLYGON ((436494.390 404739.905, 436510.960 40..."
170259,17608403B0,176,8,231,263.119995,"POLYGON ((434921.650 404586.095, 434929.200 40..."
170260,17608403B0,176,8,231,263.119995,"POLYGON ((435862.530 404798.675, 435885.390 40..."
170261,17608403B0,176,8,231,263.119995,"POLYGON ((434990.870 404323.435, 434991.530 40..."


In [23]:
wo_union = wo_old.overlay(wo_new, how='union')

AttributeError: 'GeoDataFrame' object has no attribute 'overlay'

In [24]:
gpd.__version__

'0.9.0'

In [ ]:
wo_old.union

In [8]:
wo_old.to_file('/home/philipp/Data/change_detection/GIS_data/wo_2016/wo_2016_test.shp')
wo_new.to_file('/home/philipp/Data/change_detection/GIS_data/wo_2020/wo_2020_test.shp')

### new

In [33]:
wo_union = gpd.read_file('/home/philipp/Data/change_detection/GIS_data/union/wo_union_2016_2020_test2.shp')

In [34]:
wo_union['area'] = wo_union.area

In [35]:
wo_union[wo_union['area']>1].size

839688

In [36]:
wo_union = wo_union[wo_union['area']>300]

In [37]:
wo_union = wo_union.fillna(0)

wo_union[['fb', 'fr', 'age', '2020fb', '2020fr', '2020age']] = wo_union[['fb', 'fr', 'age', '2020fb', '2020fr', '2020age']].astype(int)

In [38]:
wo_union

,WO,fb,fr,age,mass_ha,2020WO,2020fb,2020fr,2020age,2020mass_h,geometry,area
0,173060-101,173,6,0,0.0,173060-101,173,6,0,0.000000,"POLYGON ((568614.197 428967.011, 568581.040 42...",25311.227285
6,173060-101,173,6,0,0.0,173060-101,173,6,0,0.000000,"POLYGON ((563354.710 429853.135, 563355.800 42...",26726.134736
13,173060-101,173,6,0,0.0,173060-101,173,6,0,0.000000,"POLYGON ((561823.143 429919.758, 561823.140 42...",930.259792
17,173060-102,173,6,0,0.0,173060-102,173,6,0,0.000000,"POLYGON ((561313.032 423279.011, 561270.820 42...",2789.454744
19,173060-102,173,6,0,0.0,173060-102,173,6,0,0.000000,"POLYGON ((561168.274 429196.674, 561144.204 42...",995.951403
...,...,...,...,...,...,...,...,...,...,...,...,...
84465,0,0,0,0,0.0,17309399C0,173,9,104,522.450012,"MULTIPOLYGON (((592525.840 419312.023, 592525....",444.099418
84466,0,0,0,0,0.0,17309399D0,173,9,64,389.339996,"MULTIPOLYGON (((592180.080 419472.304, 592180....",576.636060
84467,0,0,0,0,0.0,17309399E1,173,9,84,388.549988,"POLYGON ((591509.007 419660.713, 591506.920 41...",409.403540
84476,0,0,0,0,0.0,1730940071,173,9,-1,0.000000,"MULTIPOLYGON (((591056.220 420875.355, 591090....",718.164219


In [39]:
wo_union_cange = wo_union[(wo_union['mass_ha']>=70) & (wo_union['2020mass_h']<40)]

In [40]:
wo_union_cange = wo_union[(wo_union['age']>=30) & (wo_union['2020age']<=15)]

In [41]:
wo_union_cange

,WO,fb,fr,age,mass_ha,2020WO,2020fb,2020fr,2020age,2020mass_h,geometry,area
81,17306122B2,173,6,45,229.250000,17306122B2,173,6,15,0.0,"MULTIPOLYGON (((561173.422 428356.863, 561170....",569.537733
133,17306122D1,173,6,160,247.250000,17306122B2,173,6,15,0.0,"POLYGON ((561220.347 428472.935, 561219.110 42...",392.802357
163,17306122F0,173,6,30,75.110001,1730612292,173,6,0,0.0,"POLYGON ((561469.130 428804.155, 561471.200 42...",2042.526003
199,17306123A1,173,6,50,333.899994,17306123A3,173,6,15,0.0,"POLYGON ((560405.370 428482.325, 560411.984 42...",709.453061
200,17306123A1,173,6,50,333.899994,17306123A4,173,6,5,0.0,"POLYGON ((560518.482 428233.760, 560517.190 42...",24297.046063
...,...,...,...,...,...,...,...,...,...,...,...,...
82047,17309404B1,173,9,97,430.250000,0,0,0,0,0.0,"MULTIPOLYGON (((588525.028 417454.348, 588523....",999.764707
82120,17309407D1,173,9,107,526.660034,0,0,0,0,0.0,"POLYGON ((585127.964 418139.334, 585128.370 41...",947.949596
82121,17309407D1,173,9,107,526.660034,0,0,0,0,0.0,"POLYGON ((585127.964 418139.334, 585128.370 41...",947.949596
82126,17309407H0,173,9,117,573.119995,0,0,0,0,0.0,"POLYGON ((585581.060 418092.776, 585580.760 41...",411.606215


In [42]:
wo_union_cange.to_file('/home/philipp/Data/change_detection/GIS_data/union/wo_union_2016_2020_filter_age.shp')